In [3]:
import tensorflow.keras as keras
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from utils import get_dendrite_preprocessed_train_data_tf
import numpy as np

/home/student401/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [25]:
X, Y = get_dendrite_preprocessed_train_data_tf()
# Get training pieces
# x 0
# 0 0
X1, Y1 = X[:, 0:100, 0:100], Y[:]
# 0 x
# 0 0
X2, Y2 = X[:, 100:200, 0:100], Y[:]
# 0 0
# x 0
X3, Y3 = X[:, 0:100, 100:200], Y[:]
# 0 0
# 0 x
X4, Y4 = X[:, 100:200, 100:200], Y[:]

# Stack all the training Xs and Ys
Xtrain, Ytrain = np.vstack([X1, X2, X3, X4]), np.vstack([Y1, Y2, Y3, Y4])

In [26]:
def rearrange_one_to_three(X):
    new_X = np.zeros((X.shape[0], 100, 100, 3))
    new_X[:, :, :, 0] = X[:, :, :, 0]
    new_X[:, :, :, 1] = X[:, :, :, 0]
    new_X[:, :, :, 2] = X[:, :, :, 0]
    return new_X

In [27]:
Xtrain = rearrange_one_to_three(Xtrain)

In [28]:
Xtrain.shape

(80000, 100, 100, 3)

In [4]:
image_size = [100, 100]

In [5]:
vgg = VGG16(input_shape=image_size + [3], weights='imagenet', include_top=False)

In [9]:
for layer in vgg.layers[0:11]:
    layer.trainable = False

In [7]:
vgg.layers[0:10]

In [17]:
X = Flatten()(vgg.output)

In [18]:
hidden_out1 = Dense(700, activation='relu')(X)

In [19]:
hidden_out2 = Dense(500, activation='relu')(hidden_out1)

In [20]:
hidden_out3 = Dense(200, activation='relu')(hidden_out2)

In [21]:
prediction = Dense(10, activation='softmax')(hidden_out3)

In [22]:
model = Model(inputs=vgg.input, outputs=prediction)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [24]:
model.compile(
    loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [40]:
r = model.fit(Xtrain, Ytrain, validation_split=0.1, epochs=1, batch_size=150)

Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 69s 964us/step - loss: 0.5580 - acc: 0.7598 - val_loss: 0.6639 - val_acc: 0.7104


In [32]:
from utils import get_dendrite_preprocessed_test_data_tf

In [33]:
Xtest, Ytest = get_dendrite_preprocessed_test_data_tf()
# Get training pieces
# x 0
# 0 0
Xtest1, Ytest1 = Xtest[:, 0:100, 0:100], Ytest[:]
# 0 x
# 0 0
Xtest2, Ytest2 = Xtest[:, 100:200, 0:100], Ytest[:]
# 0 0
# x 0
Xtest3, Ytest3 = Xtest[:, 0:100, 100:200], Ytest[:]
# 0 0
# 0 x
Xtest4, Ytest4 = Xtest[:, 100:200, 100:200], Ytest[:]

# Stack all the training Xs and Ys
Xtest, Ytest = np.vstack([Xtest1, Xtest2, Xtest3, Xtest4]), np.vstack([Ytest1, Ytest2, Ytest3, Ytest4])


In [34]:
Xtest = rearrange_one_to_three(Xtest)

In [41]:
model.evaluate(Xtest, Ytest)

20000/20000 [==============================] - 15s 736us/step


[0.6653912125587463, 0.7148]

In [36]:
np.argmax(predictions[:10], axis=1)

array([0, 2, 3, 7, 8, 2, 9, 0, 1, 5])

In [37]:
np.argmax(Ytest, axis=1)

array([0, 2, 4, ..., 2, 5, 6])

In [39]:
predictions

[0.6450756874799728, 0.73265]

In [42]:
model.save_weights('vgg_weights.hdf5')